In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import pickle
import fnmatch
import os
from sksurv.preprocessing import OneHotEncoder
%matplotlib inline
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from pprint import pprint
pd.set_option('display.width', None)
pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings('ignore')

# KM Functions

In [2]:
def build_surv_obj(survival_type, years, df_dict):
    
    """
    This function builds the survival object to be processed by kaplan meier model to return kaplan meier df
    """
    
    survival_type = str(survival_type)
    years = str(years)
    
    survival_df = df_dict[years + "_years"][survival_type]
    
    Time_df = survival_df.loc[:,[survival_type + "_days"]]
    Time_df[survival_type + "_years"] = Time_df[survival_type + "_days"]/365.25
    Time_df["status"] = survival_df["status_" + survival_type]
    Time_df.head()

    return SurvivalData(time= (survival_type+ "_years"), status="status", data=Time_df)

def KM_to_df(KM_object):
    
    # Process the summary as string
    
    summary_lines_list = str(KM_object.summary).split("\n")
    
    header = ["time", "events", "at_risk",  "estimate",  "std_error",  "95CI_lower",  "95CI_upper"]
    rows = summary_lines_list[6:]
    
    row_values = []
    
    for row in rows:
        
        elements = row.split(" ")
        tmp = []
        for element in elements:
            if element.isnumeric() or ("." in element):
                tmp.append(element)
                
        row_values.append(tmp)
        
    #Build df
    output_df = pd.DataFrame(row_values, columns=header)
                
    return output_df



# Data Processing Functions

In [3]:
def drop_by_index(X,indexes):
    """
    helper function to drop rows of dataframe and return new dataframe without those rows with indexes resetted
    """
    X = X.drop(indexes)
    X = X.reset_index().drop(columns="index")
    return(X)

def dataSetting(dropCol,FILE_FOLDER = "C:\\SMU_v2\\"):
    '''
    function to read the pkl from from datasource
        1. Remove dx_date that is NULL.
        2. Drop all rows where crucial fields for X_features are NULL.
        3. Convert Date columns into datetime format
        4. Derive OS, CSS, DFS days based on dx_date
        5. Create status column to indicate if the patient is dead or alive base on if death_age exists
    '''
    df = pd.read_pickle(FILE_FOLDER + "clinical_output.pkl").reset_index().drop(columns="index")
    to_drop = df[df['dx_date']=="NA"].index
    df = drop_by_index(df,to_drop)

    df.drop(columns=dropCol,inplace = True)

    # drop all rows where dates are null
    df.dropna(axis=0,\
                    subset=['Date_for_DFS','Date_for_OS','Date_for_CSS','dx_date','Age_@_Dx'],\
                    inplace=True)
    
    # convert all datetime in dataframe into dateime format for processing
    df["Date_for_DFS"] = pd.to_datetime(df["Date_for_DFS"])
    df["Date_for_OS"] = pd.to_datetime(df["Date_for_OS"])
    df["Date_for_CSS"] = pd.to_datetime(df["Date_for_CSS"])
    df["dx_date"] = pd.to_datetime(df["dx_date"])
    df['last_seen']= pd.to_datetime(df["dx_date"])
    df['dob']= pd.to_datetime(df["dx_date"])

    # calculate in days
    df["DFS_days"] = (df["Date_for_DFS"] - df['dx_date'] )/np.timedelta64(1, 'D')
    df["OS_days"] = (df["Date_for_OS"] - df['dx_date'] )/np.timedelta64(1, 'D')
    df["CSS_days"] = (df["Date_for_CSS"] - df['dx_date'] )/np.timedelta64(1, 'D')

    # alive or dead
    df['status'] = np.where(df['Count_as_OS'] == "dead", False, True)

    return df

def ComputeYears(df, Year_list):
    '''
    Create a list to contain df for different years of survival
    The df will filter those patient that has deceased or days of survival longer than the defined years.
    '''

    df_dict = {}

    for i in Year_list:
        tmp = {}
        for x in list(["DFS", "CSS", "OS"]):
            df['{}_{}_years'.format(x, i)] = np.where(
                                                      np.logical_or(df['death_age'] > 0,\
                                                      df['{}_days'.format(x)]/(365.25*i) >= i),\
                                                      True,False)
            tmp[x] = df[df['{}_{}_years'.format(x, i)] == True]
        df_dict['{}_years'.format(i)] = tmp
    return df_dict

def train_test(X, Y, test_size = 0.33, random_state = 42):
    '''
    Splitting the dataset into the Training set and Test set
    '''
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y,  test_size=test_size, random_state=random_state)
    
    return X_train, X_test, Y_train, Y_test

def settingXY(df, X_features, Y_features, OHE_LOCATION = "C:\\SMU_v2\\OHE\\", name=""):
    '''
    This function returns the X and Y features need for model training
        - The function also generates one pkl that contains the One Hot Encoder for new raw data 
    
    X_features = features to use for X
    Y_features = features to use for Y 
    YEAR = years of patient record interested
    SYTPE = survival type (OS, DFS, CSS)
    OHE_LOCATION = location to store the pkl file
    '''

    X = df[X_features]
    Y = df[Y_features]

    # Save enconder so that we can OHE new data
    enc = OneHotEncoder()
    enc.fit(X)
    
    # OHE for probability
    X = enc.transform(X)
    with open(OHE_LOCATION + name + '_encoder.pickle', 'wb') as f:
        pickle.dump(enc, f) 
                  
    # convert Y to structured array
    s = Y.dtypes
    Y = np.array([tuple(x) for x in Y.values], dtype=list(zip(s.index, s)))
   
    return X, Y
def layeredData(df, group_dict,y_features, YEAR, STYPE):
    
    '''
        this function generates the dataframe required for specific groups we hope to analyze
        there are total 3 different groups but group 3 consist of multiple subgroups which leads a total of 5
        dataframe.
        Group 1: patient with stage 4 cancer
        Group 2: patient which unknown records or at initial diagnosis stage
        Group 3: make up of patient that does not belong to the groups above
    '''
    model_data_dict = {}
    TO_USE = df['{}_years'.format(YEAR)][STYPE]
    
    print("Overall initial size: {} \n".format(TO_USE.shape[0]))
        
    for key,value in group_dict.items():
        TO_USE_COPY = TO_USE.copy()

        tmp = {}
        
        waves = value['wave']
    
        if key != "group 3":
            # for group 1 and group 2 select rows that contains either stage 4/non invasive in Stage
            TO_USE_COPY = TO_USE_COPY.loc[TO_USE_COPY['Stage'] == group_dict[key]['stage'][0]]
        else:
            # for group 3 do not select rows that contains either stage 4 or non invasive in c_Staging or p_Staging
            stage = np.logical_and(TO_USE_COPY['Stage'] != group_dict[key]['stage'][0],\
                                    TO_USE_COPY['Stage'] != group_dict[key]['stage'][1])
            
            TO_USE_COPY = TO_USE_COPY.loc[stage]
            
        print("{} data size: {}".format(key,len(TO_USE_COPY)))
        
        for wave in waves:
            TO_USE_COPY2 = TO_USE_COPY.copy()
            TO_USE_COPY2 = TO_USE_COPY2[waves[wave] + y_features]
            
            len_before = len(TO_USE_COPY2)
            print("\t{} data size before dropping nan: {}".format(wave,len_before))
            
            TO_USE_COPY2.dropna(axis=0,subset=waves[wave]+ y_features, inplace=True)
            TO_USE_COPY2.reset_index(drop=True)

            len_after = len(TO_USE_COPY2)
            print("\t\t after dropping nan: {}".format(len_after))
            
            for i in waves[wave]:
                if not (i in ['nodespos','Age_@_Dx','size_precise']):
                    TO_USE_COPY2.loc[:,i] = TO_USE_COPY2[i].astype("category")
                else:
                    TO_USE_COPY2.loc[:,i] = TO_USE_COPY2[i].astype("float32")
            
            X, Y = settingXY(TO_USE_COPY2, waves[wave], y_features,name= "{}_{}".format(key,wave))   
            
            TO_USE_COPY2.to_pickle("C:\\SMU_v2\\Layered Folder\\{}_{}.pkl".format(key,wave))
            
            tmp[wave] = {
                            "X": X,\
                            "Y":Y      
                        }    
    
        model_data_dict[key] = tmp
        
        
    return model_data_dict

def loadOHE(df,OHE_LOCATION = "C:\\SMU_v2\\OHE\\", name=""):
    '''
    load enconder to OHE new raw data for prediction
    '''
    with open(OHE_LOCATION + name + '_encoder.pickle', 'rb') as f:
        enc = pickle.load(f) 
    
    #type case object to category
    typeCastList = list(df.select_dtypes(include=[object]).columns)
    df[typeCastList] = df[typeCastList].astype("category")
    OHE_New_Data = enc.transform(df)
    
    return OHE_New_Data

### Data Processing

In [27]:
path = 'C:\\SMU_v2\\Layered Folder\\'

y_features = list(['status','OS_days'])

if len(fnmatch.filter(os.listdir(path), '*.pkl')) > 0:
    
    model_data_dict = {}
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.pkl' in file:
                group = str(file).split("_")[0]
                wave = str(file).split("_")[1].split(".")[0]
                tmp = pd.read_pickle(path+file)
                
                x_features = [i for i in tmp.columns if i not in y_features]
                
                X, Y = settingXY(tmp, x_features, y_features,name= "{}_{}".format(group,wave))  
                
                Y = Y.tolist()
                Y = pd.DataFrame(Y, columns=['Status','OS_days'])
                if not (group in model_data_dict): 
                    model_data_dict[group] = {wave : { "X": X,\
                                                       "Y":Y
                                                      }}
                else:
                    model_data_dict[group].update({wave : { "X": X,\
                                                            "Y":Y
                                                          }} )
                del X
                del Y
                del tmp
    print("Data loaded!")                
else:
    # Data Processing
    listToDrop = ['NRIC','dob','Has Bills?','Side','Hospital','KKH','NCCS','SGH','END_OF_ENTRY']
    clinical = dataSetting(listToDrop)
    print(clinical.shape)

    # Data of our interest are 5 and 10 years, patient that are new 
    # (does not have sufficient records will disturb and mess up our accuracy level
    # only return data that has longer timeframe than the given interval

    year_list = list([1,5,10])
    df_dict = ComputeYears(clinical,year_list)

    # Display shape of data after filtering
    for i in df_dict: 
        for s_type in df_dict[i]:
            print("Year: {}, survival category: {}, size: {}".format(i,s_type,df_dict[i][s_type].shape[0]))
            
    YEAR = 1
    STYPE = "OS"

    group_dict = { 
                    "group 1": {
                                 "stage": ['stage 4'],\
                                 'wave': {
                                             "layer 1": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2','Stage'],\
                                             "layer 2": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2',\
                                                         'T (no subgroup)', 'N (no subgroup)'],\
                                             "layer 3": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2', 'T', 'N'],\
                                             "layer 4": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2', 'size_precise', 'nodespos']
                                         }
                               },\
                    "group 2": {
                                 'stage': ['dcis/lcis non-invasive'],\
                                 'wave': {
                                             "layer 1": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2','Size'],\
                                             "layer 2": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2','size_precise']
                                         }
                               },\
                    "group 3": {
                                 "stage": ['stage 4','dcis/lcis non-invasive'],\
                                 'wave': {
                                             "layer 1": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2','Stage'],\
                                             "layer 2": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2',\
                                                         'T (no subgroup)', 'N (no subgroup)', 'M (no subgroup)'],\
                                             "layer 3": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2', 'T', 'N', 'M'],\
                                             "layer 4": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2', 'size_precise',\
                                                         'nodespos','M']
                                         }
                               },
                    }
    model_data_dict = layeredData(df_dict, group_dict,y_features,YEAR, STYPE)
    print("Processing Done!")

KeyError: "['DFS_days'] not in index"

In [24]:
pd.DataFrame(model_data_dict['group 3']['layer 1']['Y']).rename(columns={"OS_days": "time", "DFS_days":"time"})

,status,time
0,True,1961.0
1,True,2636.0
2,True,4081.0
3,True,686.0
4,True,7130.0
5,True,2449.0
6,True,1126.0
7,False,85.0
8,True,7876.0
9,True,5930.0


### Kaplan Meier Analysis

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid", palette="colorblind", color_codes=True)

from survive import datasets
from survive import SurvivalData
from survive import KaplanMeier, Breslow, NelsonAalen

Overall Survival Modeling

In [14]:
#Build Input DF
surv = build_surv_obj(survival_type="OS", years=10, df_dict=df_dict)
OS_km = KaplanMeier()
OS_km.fit(surv)

#Plot Curve
plt.figure(figsize=(10, 6))
OS_km.plot()
plt.show()
plt.close()

KeyError: 'status_OS'

In [ ]:
o = KM_to_df(OS_km)
o.to_csv("C:\\Users\\LINGXING\\Desktop\\GIT\\fyp\\Code\\km.csv",index = False)

In [ ]:
i = pd.read_csv("C:\\Users\\LINGXING\\Desktop\\GIT\\fyp\\Code\\km.csv")
i.head()

Disease Free Survival

In [12]:
#Build Input DF
surv = build_surv_obj(survival_type="DFS",years=10, df_dict=df_dict)
DFS_km = KaplanMeier()
DFS_km.fit(surv)
print(DFS_km.summary)
print(DFS_km)

#Plot curve
plt.figure(figsize=(10, 6))
DFS_km.plot()
plt.show()
plt.close()

#Estimate is basically reading off the curve at the respective time
estimate = DFS_km.predict([0.002738,1,2,3,4,5,6,7,8,9,10])
display(estimate)

KeyError: 'status_DFS'

Cancer Specific Survival

In [ ]:
#Build Input DF
surv = build_surv_obj(survival_type="CSS", years=10, df_dict=df_dict)
CSS_km = KaplanMeier()
CSS_km.fit(surv)
# print(CSS_km.summary)
print(CSS_km)

#Plot curve
plt.figure(figsize=(10, 6))
CSS_km.plot()
plt.show()
plt.close()